# Apache Spark

In this part of the notes  we look into the <a href="https://spark.apache.org/">Apache Spark</a> software. 
Apache Spark is one of the de facto tools to use when dealing with big data analytics. Specifically, we will discuss the
following items:

- How Spark applications are structured
- Performing analytics with Spark's built in functionality
- Configuring and deploying a Spark cluster

Before delving into how to use Apache Spark, let's say a few words about its architecture. This is not going to be an in depth discussion as the 
objective is to give you a high level overview.

### Apache Spark Introduction

So Spark is a popular parallel data processing framework that is written in Scala. It supports however other languages, including
Python, R, and Java, to name a few. Spark can be used as the central processing component in any data
platform, but others may be a better fit for your problem. The key thing to understand is that Spark is
separated from your storage layer, which allows you to connect Spark to any storage technology you
need. Similar tools include Flink, AWS Glue, and Snowflake.


Spark is a cluster-based in-memory processing engine that uses a master/slave approach for coordination.
The master is called the driver node, and all other computers in the clusters are worker nodes. If you
want to create a single-node cluster, the driver and worker nodes must be combined into one machine.
This setup will have issues with larger data, but there are applications where this can be useful.

A driver node will have a driver application, which coordinates the processing of your data job. Each
worker node will have an executor application. The driver and the executors will work with each other
in the parallel processing workflow, but the executor’s process data and the driver will give directions.
Each worker node/executor application has several parallelism elements that are created by counting
the number of slots. Typically, a slot can be thought of as a core or a thread. If a worker node/executor
has 10 slots, then it has 10 parallelism elements. The more you have, the more processing can be done
in parallel. There is a trade-off for everything, including cost.

The driver application will have one or more jobs for the cluster to work on. To understand a job, you
must first understand lazy evaluation. How Spark works is that all transformations are documented,
but nothing is done until an action or the driver application requests an I/O task. Actions range from
writing something to a disk to sending a message on a message system such as Kafka. This action is
called a job, and the driver will coordinate job executions. Jobs are composed of stages, and stages
are groups of tasks that can be done in parallel without the need to move data from one worker node
to another. This movement of data is called a shuffle. Again, moving data from one worker node to
another is a very expensive cost, so Spark tries to avoid it. A task is the smallest unit of work and is
done using a data partition. A partition is a parallel data unit that lives in memory on a single worker
node. Partitions are organized into a resilient distributed dataset (RDD), which is the building block
of a DataFrame.

Simply put, an RDD is a representation of data that’s not organized into any structure. Any changes that
are made to that representation are organized into tasks that get distributed across nodes. DataFrames,
on the other hand, are structured and semi-structured representations of data, but they use RDDs
behind the scene. One fundamental concept is partitions, which we will cover next.

### Partitions

When working with DataFrames, you must be mindful of how your data is partitioned across the
cluster. Some questions you might want to ask are, How many partitions is my data split into at any
given time in the job workflow? Can I have control over the number of partitions that get created?
How is my data distributed across my partitions? For example, is column X evenly distributed across
the network, or is it clumped into a small group of worker nodes? Should I increase or reduce my
partitions? Do I have control over which data gets split over which partitions?
Having your partitions closely match your number of parallelism elements is ideal. You don’t want to
have any executors not processing data, and you don’t want too many partitions, which could cause
many more of your partitions to have too little data or no data at all.


#### Shuffling partitions

Shuffling is an important concept when it comes to parallel data processing. When working with
multiple nodes to process data, the data is split across each node. When an operation is required
that moves that data from one node to another, that movement is called shuffling. In terms of data
movement, the slowest process is to move data across the network, which is why it’s often avoided.

Spark will automatically set the size of partitions to 200 after any shuffle; adjusting this number used
to be a major way to optimize your Spark job. Apache Spark introduced Adaptive Query Engine
(AQE) in version 3.2.0, which uses runtime statistics to optimize your Spark instructions. Now that
AQE is available, this setting should not be adjusted as it won’t matter.


#### Caching

Caching is the process of saving your data into memory or on the disk of the worker node for faster
retrieval. The general rule of thumb is that reusing the same DataFrame caching may be a good idea.

- Memory only: Used for storing the DataFrame in memory if the data isn’t too large:
- Memory-only sterilized: A serialized version of memory only, which translates into smaller data in memory but that is not as fast
- Memory with two other nodes: This is a final addition to the previous level, where the data is also stored in memory on two other nodes. This can be serialized

Spark also offers a version of the preceding level in memory and disk or just disk modes. Data is stored
in memory when using memory and disk modes, but if needed, disk mode is also used. It should be
noted that retrieving data from memory is always significantly faster than on disk.


### Job creation pipeline

When a job is executed on the driver program, it will go through several stages, as of Spark 3.3.0,
the latest version at the time of writing this book. Before we go through each of the main stages, let’s
explain what a plan is. A plan is a list of transformations (not related to master or worker nodes) that
must be taken for a given job.

The main stages of a jon are:

- Unresolved logic plan: We know the syntax has no issues at this point, but our plan might
still have issues. One example of issues might be references to columns with wrong names.
- Analyzed logical plan and an analyzed logical plan with a cache: Here, our unresolved logical
plan is checked against the catalog that Spark manages to see that things such as table names,
column names, and data types of columns, among others, are correct.
- Optimized logical plan: The plan is sent to the catalyst optimizer, which will take the plan and
convert it. The catalyst optimizer will look at several ways it can optimize your code, including
what goes in what stage, for example, or what order could be more effective. The most important
thing to understand is that the catalyst optimizer will take DataFrame code written in any
Spark-compatible language and output an optimized plan. One major reason why using RDDs
directly is almost always slower than DataFrames is that they will never go through the catalyst
optimizer. As a result, you’re not likely to write better RDD code.
- Spark plan: The next major step is for AQE to come in and look at our physical plan (created from
the optimized logical plan) and make drastic performance improvements. These improvements
are performed on non-streaming jobs. The five improvements are adjusting sort merges to
broadcast hash joins, adjusting partitions to an optimized state after shuffling, adjusting empty
relations, handling skew from sort merges, and shuffling hash joins.
- Selected physical plan: The most optical plan is sent to the Tungsten execution engine, which
will output even more optimized RDD code in a directed acyclic graph (DAG). This is a process
that goes from start to finish and doesn’t loop.

## References

1. Jules S. Damji, Brooke Wenig, Tathagata Das, Deny Lee, _Learning Spark. Lighting-fasts data analytics_, 2nd Edition, O'Reilly.